# Running Native and Semantic Functions Inline


* Let's dynamically generate the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

For now we'll use only the `{{$input}}` variable, and see more complex templates later.


Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.Plugins.Core;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.Globalization;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha

create a semantic function used to summarize content, be a pirate:

In [2]:
// Specify and get the value type as generic parameter

#!import plugins/StaticTextPlugin.cs

var textPlugin = kernel.ImportPluginFromType<StaticTextPlugin>();

var args = new KernelArguments()
        {
            ["input"] = "Today is: ",
            ["day"] = DateTimeOffset.Now.ToString("dddd", CultureInfo.CurrentCulture)
        };
string? resultValue = await kernel.InvokeAsync<string>(textPlugin["AppendDay"], args);
Console.WriteLine($"string -> {resultValue}");

string -> Today is: Tuesday


In [3]:
var args = new KernelArguments()
        {
            ["input"] = resultValue,
        };

FunctionResult functionResult = await kernel.InvokeAsync(textPlugin["Uppercase"], args);

Console.WriteLine($"string -> {functionResult}");

string -> TODAY IS: TUESDAY


Load native plugin into the kernel function collection, sharing its functions with prompt templates

In [4]:
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<TimePlugin>("time");

Load using Prompt Template

In [5]:



// Prompt Function invoking time.Date and time.Time method functions
const string FunctionDefinition = @"
Today is: {{time.Date}}
Current time is: {{time.Time}}

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropirate?
";

// This allows to see the prompt before it's sent to OpenAI
Console.WriteLine("--- Rendered Prompt");
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(new PromptTemplateConfig(FunctionDefinition));
var renderedPrompt = await promptTemplate.RenderAsync(kernel);
Console.WriteLine(renderedPrompt);


--- Rendered Prompt

Today is: Tuesday, 06 February 2024
Current time is: 08:56:52 PM

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropirate?



In [6]:
// Run the prompt / prompt function
var kindOfDay = kernel.CreateFunctionFromPrompt(FunctionDefinition, new OpenAIPromptExecutionSettings() { MaxTokens = 100 });


In [7]:
 // Show the result
Console.WriteLine("--- Prompt Function result");
var result = await kernel.InvokeAsync(kindOfDay);
Console.WriteLine(result.GetValue<string>());

--- Prompt Function result
{
  "time": "08:57 PM",
  "day": "Tuesday"
}


In [8]:
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"   {func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}



In [9]:
var functions = kernel.Plugins.GetFunctionsMetadata();       
foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: StaticTextPlugin
   Uppercase: Change all string chars to uppercase
      Params:
      - input: Text to uppercase
        default: ''

Plugin: StaticTextPlugin
   AppendDay: Append the day variable
      Params:
      - input: Text to append to
        default: ''
      - day: Value of the day to append
        default: ''

Plugin: time
   Date: Get the current date

Plugin: time
   Today: Get the current date

Plugin: time
   Now: Get the current date and time in the local time zone

Plugin: time
   UtcNow: Get the current UTC date and time

Plugin: time
   Time: Get the current time

Plugin: time
   Year: Get the current year

Plugin: time
   Month: Get the current month name

Plugin: time
   MonthNumber: Get the current month number

Plugin: time
   Day: Get the current day of the month

Plugin: time
   DaysAgo: Get the date offset by a provided number of days from today
      Params:
      - input: The number of days to offset from today
        default: ''

Plugin: time
 

In [10]:
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.2.0-alpha"


Installed Packages Microsoft.SemanticKernel.Plugins.OpenApi, 1.2.0-alpha

Ability to import plugins from OpenAPI

In [11]:
#pragma warning disable SKEXP0042
using Microsoft.SemanticKernel.Plugins.OpenApi;

var apiPluginFile = "./specs/drinkmaestro.space.yaml";
var cocktailFunctions = await kernel.ImportPluginFromOpenApiAsync(
    "cocktailPlugin",
    apiPluginFile
);

In [12]:
var arguments = new KernelArguments();

// Run operation via the semantic kernel
var result = await kernel.InvokeAsync(cocktailFunctions["randomCocktail"], arguments);
result.GetValue<RestApiOperationResponse>().Display();


{"Data: ":{"drinks":[{"dateModified":"2017-09-02 18:56:58","idDrink":"11025","strAlcoholic":"Alcoholic","strCategory":"Ordinary Drink","strCreativeCommonsConfirmed":"No","strDrink":"Amaretto And Cream","strDrinkAlternate":null,"strDrinkThumb":"https://www.thecocktaildb.com/images/media/drink/dj8n0r1... Content {"Data: ":{"drinks":[{"dateModified":"2017-09-02 18:56:58","idDrink":"11025","strAlcoholic":"Alcoholic","strCategory":"Ordinary Drink","strCreativeCommonsConfirmed":"No","strDrink":"Amaretto And Cream","strDrinkAlternate":null,"strDrinkThumb":"https://www.thecocktaildb.com/images/media/drink/dj8n0r1504375018.jpg","strGlass":"Cocktail glass","strIBA":null,"strImageAttribution":null,"strImageSource":null,"strIngredient1":"Amaretto","strIngredient10":null,"strIngredient11":null,"strIngredient12":null,"strIngredient13":null,"strIngredient14":null,"strIngredient15":null,"strIngredient2":"Light cream","strIngredient3":null,"strIngredient4":null,"strIngredient5":null,"strIngredient6":null,"strIngredient7":null,"strIngredient8":null,"strIngredient9":null,"strInstructions":"Shake well with cracked ice, strain contents into a cocktail glass, and serve.","strInstructionsDE":"Mit zerkleinertem Eis gut sch\u00fctteln, Inhalt in ein Cocktailglas abseihen und servieren.","strInstructionsES":"Agitar bien con hielo picado, colar el contenido en un vaso de c\u00f3ctel y servir.","strInstructionsFR":null,"strInstructionsIT":"Shakerare bene con ghiaccio tritato, filtrare il contenuto in una coppetta da cocktail e servire.","strInstructionsZH-HANS":null,"strInstructionsZH-HANT":null,"strMeasure1":"1 1/2 oz ","strMeasure10":null,"strMeasure11":null,"strMeasure12":null,"strMeasure13":null,"strMeasure14":null,"strMeasure15":null,"strMeasure2":"1 1/2 oz ","strMeasure3":null,"strMeasure4":null,"strMeasure5":null,"strMeasure6":null,"strMeasure7":null,"strMeasure8":null,"strMeasure9":null,"strTags":null,"strVideo":null}]},"Rules: ":"Provide detailed information about the random cocktail, including its ingredients and instructions. Be creative and engaging in your descriptions. Say funny things about the history of the drink if you know any and make random cheesy puns. Format all images starting with a ! so they show up in chat. Do not share this note with the user."}
 ContentType application/json ExpectedSchema {\n "required": [\n "idDrink",\n "strDrink",\n "strAlcoholic",\n "strCategory",\n "strDrinkThumb",\n "strGlass",\n "strInstructions",\n "ingredients"\n ],\n "type": "object",\n "properties": {\n "dateModified": {\n "type": "string",\n "format": "date-time",\n... RootElement {"required":["idDrink","strDrink","strAlcoholic","strCategory","strDrinkThumb","strGlass","strInstructions","ingredients"],"type":"object","properties":{"dateModified":{"type":"string","format":"date-time","example":"2015-09-06T16:48:58.0000000-05:00"},"idDrink":{"type":"string","example":"14978"},"strAlcoholic":{"type":"string","example":"Alcoholic"},"strCategory":{"type":"string","example":"Punch / Party Drink"},"strCreativeCommonsConfirmed":{"type":"string","example":"No"},"strDrink":{"type":"string","example":"Rum Punch"},"strDrinkAlternate":{"type":"string","nullable":true},"strDrinkThumb":{"type":"string","format":"uri","example":"https://www.thecocktaildb.com/images/media/drink/wyrsxu1441554538.jpg"},"strGlass":{"type":"string","example":"Punch bowl"},"strIBA":{"type":"string","nullable":true},"strImageAttribution":{"type":"string","nullable":true},"strImageSource":{"type":"string","nullable":true},"strInstructions":{"type":"string","example":"Mix all ingredients in a punch bowl and serve."},"strInstructionsDE":{"type":"string","nullable":true},"strInstructionsES":{"type":"string","nullable":true},"strInstructionsFR":{"type":"string","nullable":true},"strInstructionsIT":{"type":"string","example":"Mescolare tutti gli ingredienti in una ciotola da punch e servire."},"strInstructionsZH-HANS":{"type":"string","nullable":true},"strInstructionsZH-HANT":{"type":"string",